# Peer-graded Assignment: Capstone Project

# The Battle of Neighborhoods (Week 2) 

## Introduction/Business Problem

In this project I will try to find what is the __best metro station__ to stop if I want to go for an __ice cream__  and then either enjoy the sun in a nice __outdoor space__ or a instead go to a __museum__, in __Lisbon__, __Portugal__.

Because I may be limited in terms of FourSquare quota, I only want to check the __red__ and __green__ lines.

This would be of interest to any potential tourist that uses the metro in Lisbon or even a local that just wants to have a nice time!

The best spot will be chosen based on the variety and quantity of options available in each station.

## Data

### Libraries

A number of libraries will be needed in this project.

In [49]:
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
import folium
import math
import requests
import json # library to handle JSON files
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported.')

Libraries imported.


### Lisbon Metro Stations 

Lisbon Metro has 4 lines (Green, Blue, Yellow and Red) and 50 stations, 6 of them being double stations for two lines:
1. Alameda (Red and Green Line)
2. Marques de Pombal (Blue and Yellow Line)
3. Campo Grande (Green and Yellow Line)
4. Baixa-Chiado (Blue and Green Line)
5. Sao Sebastiao (Blue and Red Line)
6. Saldanha (Red And Yellow Line)

Let's import the coordinates of all of the stations (csv file in my github area).

In [2]:
url = "https://raw.githubusercontent.com/AnaMariaAnaMaria/Coursera_Capstone/master/MetroLx.csv"
headers = ["Abbrevation","Name","Line","Latitude","Longitude"]
coordinates = pd.read_csv(url,names = headers,skiprows=1)
coordinates

,Abbrevation,Name,Line,Latitude,Longitude
0,AE,Areeiro,green,38.742222,-9.134167
1,AF,Alfornelos,blue,38.760278,-9.205000
2,AH,Alto dos Moinhos,blue,38.749444,-9.179444
3,AL,Alvalade,green,38.753333,-9.143889
4,AM,Alameda,green,38.736667,-9.133889
5,AM,Alameda,red,38.736667,-9.133889
6,AN,Anjos,green,38.726111,-9.134722
7,AP,Aeroporto,red,38.768611,-9.128611
8,AR,Arroios,green,38.733056,-9.133889
9,AS,Amadora Este,blue,38.757778,-9.218056


Let's create a map of Lisbon with all of the stations and their respective line (colour).
Please note that if the map is not visible, it can also be seen in https://github.com/AnaMariaAnaMaria/Coursera_Capstone/blob/master/LisbonMetroStations.JPG.

In [4]:
#lisbon coordinates:
lx_lat = 38.736946
lx_long = -9.142685

# create map
map_lx = folium.Map(location=[lx_lat, lx_long], zoom_start=12)

# add markers to the map
for lat, lon, poi, line in zip(coordinates['Latitude'], coordinates['Longitude'],coordinates['Name'],coordinates['Line']):
    label = folium.Popup(str(poi)+' ('+str(line)+' line)' , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=line,
        fill=True,
        fill_color=line,
        fill_opacity=0.7).add_to(map_lx)

map_lx

Because I may be limited in terms of FourSquare quota, I am just going to be looking at the __Red__ and __Green__ lines.

Also, Alameda is both on the red and on the green line, so let's drop one of the rows. And drop the Line column too.

In [5]:
two_lines = coordinates[coordinates.Line != 'blue']
two_lines = two_lines[two_lines.Line != 'yellow']
two_lines = two_lines[(two_lines.Line != 'red') | (two_lines.Abbrevation != 'AM')]
two_lines.drop(['Line'], axis = 1,inplace = True) 
two_lines = two_lines.reset_index(drop = True)
two_lines

,Abbrevation,Name,Latitude,Longitude
0,AE,Areeiro,38.742222,-9.134167
1,AL,Alvalade,38.753333,-9.143889
2,AM,Alameda,38.736667,-9.133889
3,AN,Anjos,38.726111,-9.134722
4,AP,Aeroporto,38.768611,-9.128611
5,AR,Arroios,38.733056,-9.133889
6,BC,Baixa-Chiado,38.710556,-9.139444
7,BV,Bela Vista,38.746667,-9.116944
8,CG,Campo Grande,38.760000,-9.157778
9,CH,Chelas,38.754167,-9.113611


In [6]:
two_lines.shape


(24, 4)

The final data set has 24 metro stations.

#### Finding Venues in FourSquare

Let's use FourSquare to find what venues exist near each green or red line station.

My credentials are loaded from a json file.

In [7]:
#credentials as per suggestion in https://www.coursera.org/learn/applied-data-science-capstone/discussions/weeks/3/threads/VCjKK35VEemkuBJz3kVAHA
secrets = json.load(open('secrets.json'))
CLIENT_ID = secrets['CLIENT_ID']
CLIENT_SECRET = secrets['CLIENT_SECRET']
VERSION = secrets['VERSION']

print('Credentials loaded')

Credentials loaded


I am only interested in specific categories (from https://developer.foursquare.com/docs/resources/categories):

In [66]:
venues_cat = [
 '4bf58dd8d48988d1c9941735' # Ice Cream Shop
,'4bf58dd8d48988d165941735' # Scenic Lookout
,'4bf58dd8d48988d15a941735' # Garden
,'4bf58dd8d48988d163941735' # Park
,'4bf58dd8d48988d1e2931735' # Art Gallery
,'4bf58dd8d48988d190941735' # History Museum
,'4bf58dd8d48988d18f941735' # Art Museum
,'4fceea171983d5d06c3e9823' # Aquarium
,'50aaa49e4b90af0d42d5de11' # Castle
,'4bf58dd8d48988d191941735' # Science Museum
,'4bf58dd8d48988d15b941735' # Farm
,'4bf58dd8d48988d166941735' # Sculpture Garden
,'56aa371be4b08b9a8d5734db' # Amphitheater
,'4bf58dd8d48988d1e2941735' # Beach
,'4bf58dd8d48988d181941735' # Museum
]


In [9]:
# adapted function that extracts nearby venues from the lab week 3
def getNearbyVenues(names, latitudes, longitudes, radius=1000,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['id'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Metro Station', 
                  'Metro Station Latitude', 
                  'Metro Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category Id',
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
lx_venues = getNearbyVenues(names=two_lines['Name'],
                                   latitudes=two_lines['Latitude'],
                                   longitudes=two_lines['Longitude']
                                  )

Areeiro
Alvalade
Alameda
Anjos
Aeroporto
Arroios
Baixa-Chiado
Bela Vista
Campo Grande
Chelas
Cabo Ruivo
Cais do Sodre
Encarnacao
Intendente
Martim Moniz
Moscavide
Olaias
Oriente
Olivais
Roma
Rossio
Saldanha
Sao Sebastiao
Telheiras


Let's save the list into a csv file.

In [11]:
lx_venues.to_csv('test.csv', sep='\t', encoding='utf-8')

The list of venues has 2,003 elements.


In [12]:
lx_venues.shape

(2003, 8)

I am only interested in the categories mentioned above - venues_cat.

There are 138 venues in these categories.

In [13]:
lx_venues_new = lx_venues[lx_venues['Venue Category Id'].isin(venues_cat)]
lx_venues_new.shape

(138, 8)

In [14]:
lx_venues_new.reset_index(drop=True)

,Metro Station,Metro Station Latitude,Metro Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category Id,Venue Category
0,Areeiro,38.742222,-9.134167,FIB - il vero gelato italiano (geladosfib),38.744250,-9.134210,4bf58dd8d48988d1c9941735,Ice Cream Shop
1,Areeiro,38.742222,-9.134167,Jardim Fernando Pessa,38.743069,-9.137231,4bf58dd8d48988d15a941735,Garden
2,Areeiro,38.742222,-9.134167,Parque da Fonte Luminosa,38.737068,-9.132833,4bf58dd8d48988d163941735,Park
3,Areeiro,38.742222,-9.134167,Casa do Gelado,38.744872,-9.139545,4bf58dd8d48988d1c9941735,Ice Cream Shop
4,Areeiro,38.742222,-9.134167,Culturgest,38.740828,-9.142939,4bf58dd8d48988d1e2931735,Art Gallery
5,Areeiro,38.742222,-9.134167,Surf Gelados,38.739372,-9.137052,4bf58dd8d48988d1c9941735,Ice Cream Shop
6,Areeiro,38.742222,-9.134167,La Fabrica,38.736899,-9.141959,4bf58dd8d48988d1c9941735,Ice Cream Shop
7,Areeiro,38.742222,-9.134167,Jardim do Arco do Cego,38.735908,-9.142256,4bf58dd8d48988d15a941735,Garden
8,Alvalade,38.753333,-9.143889,Gelados Conchanata,38.753678,-9.142381,4bf58dd8d48988d1c9941735,Ice Cream Shop
9,Alvalade,38.753333,-9.143889,Jardim do Campo Grande,38.756770,-9.153823,4bf58dd8d48988d163941735,Park


Let's see all the venues on a map:

In [15]:
# create map
map_lx = folium.Map(location=[lx_lat, lx_long], zoom_start=12)

# add markers to the map
for lat, lon, poi in zip(lx_venues_new['Venue Latitude'], lx_venues_new['Venue Longitude'],lx_venues_new['Venue'],):
    label = folium.Popup(str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_lx)

map_lx

Please note that if the map is not visible, it can also be seen in https://github.com/AnaMariaAnaMaria/Coursera_Capstone/blob/master/LisbonVenues.JPG

## Methodology 

### Analyzing each metro station

In [37]:
# one hot encoding
lx_venues_new_onehot = pd.get_dummies(lx_venues_new[['Venue Category']], prefix="", prefix_sep="")


## add metro station Longitude column back to dataframe
lx_venues_new_onehot['Metro Station Longitude'] = lx_venues_new['Metro Station Longitude'] 

# move metro station Longitude column to the first column
fixed_columns = [lx_venues_new_onehot.columns[-1]] + list(lx_venues_new_onehot.columns[:-1])
lx_venues_new_onehot = lx_venues_new_onehot[fixed_columns]


## add metro station Latitude column back to dataframe
lx_venues_new_onehot['Metro Station Latitude'] = lx_venues_new['Metro Station Latitude'] 

# move metro station Latitude column to the first column
fixed_columns = [lx_venues_new_onehot.columns[-1]] + list(lx_venues_new_onehot.columns[:-1])
lx_venues_new_onehot = lx_venues_new_onehot[fixed_columns]


## add metro station column back to dataframe
lx_venues_new_onehot['Metro Station'] = lx_venues_new['Metro Station'] 

# move metro station column to the first column
fixed_columns = [lx_venues_new_onehot.columns[-1]] + list(lx_venues_new_onehot.columns[:-1])
lx_venues_new_onehot = lx_venues_new_onehot[fixed_columns]


lx_venues_new_onehot.head()


,Metro Station,Metro Station Latitude,Metro Station Longitude,Amphitheater,Aquarium,Art Gallery,Art Museum,Beach,Castle,Farm,Garden,History Museum,Ice Cream Shop,Museum,Park,Scenic Lookout,Science Museum,Sculpture Garden
0,Areeiro,38.742222,-9.134167,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,Areeiro,38.742222,-9.134167,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
6,Areeiro,38.742222,-9.134167,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
11,Areeiro,38.742222,-9.134167,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
24,Areeiro,38.742222,-9.134167,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
lx_venues_new_onehot.shape

(138, 18)

Let's group all the venues per metro station:

In [92]:
lx_venues_new_grouped = lx_venues_new_onehot.groupby(['Metro Station','Metro Station Latitude','Metro Station Longitude']).sum().reset_index()
lx_venues_new_grouped

,Metro Station,Metro Station Latitude,Metro Station Longitude,Amphitheater,Aquarium,Art Gallery,Art Museum,Beach,Castle,Farm,Garden,History Museum,Ice Cream Shop,Museum,Park,Scenic Lookout,Science Museum,Sculpture Garden
0,Alameda,38.736667,-9.133889,0,0,1,0,0,0,0,3,0,3,0,1,1,0,0
1,Alvalade,38.753333,-9.143889,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
2,Anjos,38.726111,-9.134722,0,0,0,0,0,0,0,2,0,1,0,1,3,0,0
3,Areeiro,38.742222,-9.134167,0,0,1,0,0,0,0,2,0,4,0,1,0,0,0
4,Arroios,38.733056,-9.133889,0,0,0,0,1,0,0,1,0,3,0,1,2,0,0
5,Baixa-Chiado,38.710556,-9.139444,0,0,0,0,0,0,0,0,1,6,0,0,1,0,0
6,Bela Vista,38.746667,-9.116944,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
7,Cabo Ruivo,38.762500,-9.103889,0,1,0,0,0,0,1,0,0,3,0,0,1,1,0
8,Cais do Sodre,38.705278,-9.142500,0,0,1,0,0,0,0,0,0,4,0,0,1,0,0
9,Campo Grande,38.760000,-9.157778,0,0,0,1,0,0,0,1,2,1,0,1,0,0,0


We consider only Metro Stations that have at least one ice cream shop in their area.

In [93]:
lx_venues_new_grouped = lx_venues_new_grouped[lx_venues_new_grouped['Ice Cream Shop'] > 0].reset_index(drop=True)
lx_venues_new_grouped

,Metro Station,Metro Station Latitude,Metro Station Longitude,Amphitheater,Aquarium,Art Gallery,Art Museum,Beach,Castle,Farm,Garden,History Museum,Ice Cream Shop,Museum,Park,Scenic Lookout,Science Museum,Sculpture Garden
0,Alameda,38.736667,-9.133889,0,0,1,0,0,0,0,3,0,3,0,1,1,0,0
1,Alvalade,38.753333,-9.143889,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
2,Anjos,38.726111,-9.134722,0,0,0,0,0,0,0,2,0,1,0,1,3,0,0
3,Areeiro,38.742222,-9.134167,0,0,1,0,0,0,0,2,0,4,0,1,0,0,0
4,Arroios,38.733056,-9.133889,0,0,0,0,1,0,0,1,0,3,0,1,2,0,0
5,Baixa-Chiado,38.710556,-9.139444,0,0,0,0,0,0,0,0,1,6,0,0,1,0,0
6,Cabo Ruivo,38.762500,-9.103889,0,1,0,0,0,0,1,0,0,3,0,0,1,1,0
7,Cais do Sodre,38.705278,-9.142500,0,0,1,0,0,0,0,0,0,4,0,0,1,0,0
8,Campo Grande,38.760000,-9.157778,0,0,0,1,0,0,0,1,2,1,0,1,0,0,0
9,Intendente,38.722500,-9.135000,0,0,0,0,0,0,0,2,0,1,0,1,3,0,0


The subset is narrowed to 18 stations.

Let's add a total column with the total number of venues near each station and also the total of venues that are outdoors or museums.

In [94]:
outdoors = ['Scenic Lookout','Garden','Park','Castle','Farm','Sculpture Garden','Amphitheater','Beach']
museums = ['Art Gallery','History Museum','Art Museum','Aquarium','Science Museum','Museum']
total = outdoors + museums + ['Ice Cream Shop']

lx_venues_new_grouped['Total Outdoors'] = lx_venues_new_grouped[outdoors].sum(axis=1)

lx_venues_new_grouped['Total Museums'] = lx_venues_new_grouped[museums].sum(axis=1)

lx_venues_new_grouped['Total'] = lx_venues_new_grouped[total].sum(axis=1)

lx_venues_new_grouped

,Metro Station,Metro Station Latitude,Metro Station Longitude,Amphitheater,Aquarium,Art Gallery,Art Museum,Beach,Castle,Farm,...,History Museum,Ice Cream Shop,Museum,Park,Scenic Lookout,Science Museum,Sculpture Garden,Total Outdoors,Total Museums,Total
0,Alameda,38.736667,-9.133889,0,0,1,0,0,0,0,...,0,3,0,1,1,0,0,5,1,9
1,Alvalade,38.753333,-9.143889,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,2
2,Anjos,38.726111,-9.134722,0,0,0,0,0,0,0,...,0,1,0,1,3,0,0,6,0,7
3,Areeiro,38.742222,-9.134167,0,0,1,0,0,0,0,...,0,4,0,1,0,0,0,3,1,8
4,Arroios,38.733056,-9.133889,0,0,0,0,1,0,0,...,0,3,0,1,2,0,0,5,0,8
5,Baixa-Chiado,38.710556,-9.139444,0,0,0,0,0,0,0,...,1,6,0,0,1,0,0,1,1,8
6,Cabo Ruivo,38.762500,-9.103889,0,1,0,0,0,0,1,...,0,3,0,0,1,1,0,2,2,7
7,Cais do Sodre,38.705278,-9.142500,0,0,1,0,0,0,0,...,0,4,0,0,1,0,0,1,1,6
8,Campo Grande,38.760000,-9.157778,0,0,0,1,0,0,0,...,2,1,0,1,0,0,0,2,3,6
9,Intendente,38.722500,-9.135000,0,0,0,0,0,0,0,...,0,1,0,1,3,0,0,6,0,7


### Creating clusters of Metro Stations

#### Run *k*-means to cluster the Metro Stations into 4 clusters.

In [95]:
# set number of clusters
kclusters = 4

lx_venues_new_grouped_clustering = lx_venues_new_grouped.drop('Metro Station', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lx_venues_new_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 2, 3, 2, 3, 3, 3, 1, 2])

Let's add the cluster label to the original data set.

In [96]:
# add clustering labels
lx_venues_new_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

lx_venues_new_grouped


,Cluster Labels,Metro Station,Metro Station Latitude,Metro Station Longitude,Amphitheater,Aquarium,Art Gallery,Art Museum,Beach,Castle,...,History Museum,Ice Cream Shop,Museum,Park,Scenic Lookout,Science Museum,Sculpture Garden,Total Outdoors,Total Museums,Total
0,2,Alameda,38.736667,-9.133889,0,0,1,0,0,0,...,0,3,0,1,1,0,0,5,1,9
1,1,Alvalade,38.753333,-9.143889,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,2
2,2,Anjos,38.726111,-9.134722,0,0,0,0,0,0,...,0,1,0,1,3,0,0,6,0,7
3,3,Areeiro,38.742222,-9.134167,0,0,1,0,0,0,...,0,4,0,1,0,0,0,3,1,8
4,2,Arroios,38.733056,-9.133889,0,0,0,0,1,0,...,0,3,0,1,2,0,0,5,0,8
5,3,Baixa-Chiado,38.710556,-9.139444,0,0,0,0,0,0,...,1,6,0,0,1,0,0,1,1,8
6,3,Cabo Ruivo,38.762500,-9.103889,0,1,0,0,0,0,...,0,3,0,0,1,1,0,2,2,7
7,3,Cais do Sodre,38.705278,-9.142500,0,0,1,0,0,0,...,0,4,0,0,1,0,0,1,1,6
8,1,Campo Grande,38.760000,-9.157778,0,0,0,1,0,0,...,2,1,0,1,0,0,0,2,3,6
9,2,Intendente,38.722500,-9.135000,0,0,0,0,0,0,...,0,1,0,1,3,0,0,6,0,7


Let's see our clusters in a map:

In [135]:
# create map
map_clusters = folium.Map(location=[lx_lat, lx_long], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, nic, out, mus in zip(
         lx_venues_new_grouped['Metro Station Latitude']
        ,lx_venues_new_grouped['Metro Station Longitude']
        ,lx_venues_new_grouped['Metro Station']
        ,lx_venues_new_grouped['Cluster Labels']
        ,lx_venues_new_grouped['Ice Cream Shop']
        ,lx_venues_new_grouped['Total Outdoors']
        ,lx_venues_new_grouped['Total Museums']
       ):
    label = folium.Popup(
            str(poi) 
            + ' Cluster ' + str(cluster) 
            + ' Number of Ice Cream Shops: ' + str(nic)
            + ' Number of Outdoors: ' + str(out)
            + ' Number of Museums: ' + str(mus)
            , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Results and Discussion

Let's look at our 4 clusters.

### Cluster 1 - Ice Cream and the Castle

This cluster has two metro stations, __Martim Moniz__ and __Rossio__, each with at least 3 __Ice Cream Shops__ and they are both close to the __Castle__. These two stations are the ones that have the __higher number of venues__ closeby (12 and 11).

In [98]:
lx_venues_new_grouped.loc[lx_venues_new_grouped['Cluster Labels'] == 0]

,Cluster Labels,Metro Station,Metro Station Latitude,Metro Station Longitude,Amphitheater,Aquarium,Art Gallery,Art Museum,Beach,Castle,...,History Museum,Ice Cream Shop,Museum,Park,Scenic Lookout,Science Museum,Sculpture Garden,Total Outdoors,Total Museums,Total
10,0,Martim Moniz,38.716944,-9.135556,0,0,0,0,0,1,...,1,3,0,0,5,0,0,8,1,12
14,0,Rossio,38.713611,-9.138056,0,0,0,0,0,1,...,1,6,0,0,3,0,0,4,1,11


Let's have a look at the __ice cream shops__.

Rossio has a bit more variety, and actually two of the shops are the same (these two metro stations are very close to each other).

In [125]:
cluster1 = lx_venues_new_grouped.loc[lx_venues_new_grouped['Cluster Labels'] == 0]['Metro Station'].tolist()
lx_venues_new[lx_venues_new['Metro Station'].isin(cluster1) & lx_venues_new['Venue Category'].isin(['Ice Cream Shop'])]

,Metro Station,Metro Station Latitude,Metro Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category Id,Venue Category
1176,Martim Moniz,38.716944,-9.135556,Amorino,38.711807,-9.138144,4bf58dd8d48988d1c9941735,Ice Cream Shop
1204,Martim Moniz,38.716944,-9.135556,Santini,38.711138,-9.139668,4bf58dd8d48988d1c9941735,Ice Cream Shop
1209,Martim Moniz,38.716944,-9.135556,Mú Gelato Italiano,38.722395,-9.140521,4bf58dd8d48988d1c9941735,Ice Cream Shop
1619,Rossio,38.713611,-9.138056,Amorino,38.711807,-9.138144,4bf58dd8d48988d1c9941735,Ice Cream Shop
1636,Rossio,38.713611,-9.138056,Santini,38.711138,-9.139668,4bf58dd8d48988d1c9941735,Ice Cream Shop
1651,Rossio,38.713611,-9.138056,Amorino Chiado,38.710808,-9.140747,4bf58dd8d48988d1c9941735,Ice Cream Shop
1669,Rossio,38.713611,-9.138056,Grom,38.710913,-9.140544,4bf58dd8d48988d1c9941735,Ice Cream Shop
1695,Rossio,38.713611,-9.138056,Fragoleto,38.709725,-9.136435,4bf58dd8d48988d1c9941735,Ice Cream Shop
1705,Rossio,38.713611,-9.138056,Gelato Therapy,38.710149,-9.135339,4bf58dd8d48988d1c9941735,Ice Cream Shop


Let's have a look at the __outdoors__.

Other than the Castle, there are many Scenic Lookouts and Gardens in the area. Perfect for a sunny day!

In [126]:
lx_venues_new[lx_venues_new['Metro Station'].isin(cluster1) & lx_venues_new['Venue Category'].isin(outdoors)]

,Metro Station,Metro Station Latitude,Metro Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category Id,Venue Category
1114,Martim Moniz,38.716944,-9.135556,Miradouro da Senhora do Monte,38.719072,-9.132793,4bf58dd8d48988d165941735,Scenic Lookout
1116,Martim Moniz,38.716944,-9.135556,Miradouro Sophia de Mello Breyner Andresen,38.716454,-9.131557,4bf58dd8d48988d165941735,Scenic Lookout
1119,Martim Moniz,38.716944,-9.135556,Jardim da Cerca da Graça,38.717911,-9.132196,4bf58dd8d48988d15a941735,Garden
1149,Martim Moniz,38.716944,-9.135556,Miradouro do Castelo de São Jorge,38.712525,-9.133070,4bf58dd8d48988d165941735,Scenic Lookout
1154,Martim Moniz,38.716944,-9.135556,Castelo de São Jorge,38.712480,-9.133103,50aaa49e4b90af0d42d5de11,Castle
1156,Martim Moniz,38.716944,-9.135556,Jardim do Torel,38.718980,-9.141523,4bf58dd8d48988d15a941735,Garden
1191,Martim Moniz,38.716944,-9.135556,Terraços do Carmo,38.712332,-9.139709,4bf58dd8d48988d165941735,Scenic Lookout
1200,Martim Moniz,38.716944,-9.135556,Miradouro de Santa Luzia,38.711685,-9.130207,4bf58dd8d48988d165941735,Scenic Lookout
1631,Rossio,38.713611,-9.138056,Terraços do Carmo,38.712332,-9.139709,4bf58dd8d48988d165941735,Scenic Lookout
1661,Rossio,38.713611,-9.138056,Miradouro do Castelo de São Jorge,38.712525,-9.133070,4bf58dd8d48988d165941735,Scenic Lookout


### Cluster 2 - Mix of Museums and Outdoors

Cluster two has 5 metro stations, __Alvalade__, __Campo Grande__, __Roma__, __Saldanha__ and __Telheiras__. There is not the same variety in terms of Ice Cream Shops but there is a __good mix of Museums and Outdoors__.

In [99]:
lx_venues_new_grouped.loc[lx_venues_new_grouped['Cluster Labels'] == 1]

,Cluster Labels,Metro Station,Metro Station Latitude,Metro Station Longitude,Amphitheater,Aquarium,Art Gallery,Art Museum,Beach,Castle,...,History Museum,Ice Cream Shop,Museum,Park,Scenic Lookout,Science Museum,Sculpture Garden,Total Outdoors,Total Museums,Total
1,1,Alvalade,38.753333,-9.143889,0,0,0,0,0,0,...,0,1,0,1,0,0,0,1,0,2
8,1,Campo Grande,38.760000,-9.157778,0,0,0,1,0,0,...,2,1,0,1,0,0,0,2,3,6
13,1,Roma,38.748889,-9.141389,0,0,1,0,0,0,...,0,3,0,0,0,0,0,1,1,5
15,1,Saldanha,38.735278,-9.144167,0,0,2,1,0,0,...,0,1,0,0,0,0,1,2,3,6
17,1,Telheiras,38.760556,-9.165000,0,0,0,1,0,0,...,2,1,0,1,0,0,0,2,3,6


Let's have a look at the __ice cream shops__.

Roma has a bit more variety and one of the shops is shared with Alvalade, which is very closeby.

In [127]:
cluster2 = lx_venues_new_grouped.loc[lx_venues_new_grouped['Cluster Labels'] == 1]['Metro Station'].tolist()
lx_venues_new[lx_venues_new['Metro Station'].isin(cluster2) & lx_venues_new['Venue Category'].isin(['Ice Cream Shop'])]

,Metro Station,Metro Station Latitude,Metro Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category Id,Venue Category
100,Alvalade,38.753333,-9.143889,Gelados Conchanata,38.753678,-9.142381,4bf58dd8d48988d1c9941735,Ice Cream Shop
678,Campo Grande,38.760000,-9.157778,Geladaria Artisani,38.760716,-9.168041,4bf58dd8d48988d1c9941735,Ice Cream Shop
1521,Roma,38.748889,-9.141389,Gelados Conchanata,38.753678,-9.142381,4bf58dd8d48988d1c9941735,Ice Cream Shop
1526,Roma,38.748889,-9.141389,Casa do Gelado,38.744872,-9.139545,4bf58dd8d48988d1c9941735,Ice Cream Shop
1551,Roma,38.748889,-9.141389,FIB - il vero gelato italiano (geladosfib),38.744250,-9.134210,4bf58dd8d48988d1c9941735,Ice Cream Shop
1738,Saldanha,38.735278,-9.144167,La Fabrica,38.736899,-9.141959,4bf58dd8d48988d1c9941735,Ice Cream Shop
1916,Telheiras,38.760556,-9.165000,Geladaria Artisani,38.760716,-9.168041,4bf58dd8d48988d1c9941735,Ice Cream Shop


Let's have a look at the __outdoors__.

There is a mix of Gardens and Parks.

In [128]:
lx_venues_new[lx_venues_new['Metro Station'].isin(cluster2) & lx_venues_new['Venue Category'].isin(outdoors)]

,Metro Station,Metro Station Latitude,Metro Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category Id,Venue Category
143,Alvalade,38.753333,-9.143889,Jardim do Campo Grande,38.756770,-9.153823,4bf58dd8d48988d163941735,Park
656,Campo Grande,38.760000,-9.157778,Jardim do Campo Grande,38.756770,-9.153823,4bf58dd8d48988d163941735,Park
740,Campo Grande,38.760000,-9.157778,Jardim Prof António De Sousa Franco,38.760573,-9.168868,4bf58dd8d48988d15a941735,Garden
1542,Roma,38.748889,-9.141389,Jardim Fernando Pessa,38.743069,-9.137231,4bf58dd8d48988d15a941735,Garden
1734,Saldanha,38.735278,-9.144167,Jardim do Arco do Cego,38.735908,-9.142256,4bf58dd8d48988d15a941735,Garden
1792,Saldanha,38.735278,-9.144167,Jardim da Fundação Calouste Gulbenkian,38.736794,-9.153515,4bf58dd8d48988d166941735,Sculpture Garden
1984,Telheiras,38.760556,-9.165000,Jardim do Melkia,38.763798,-9.171710,4bf58dd8d48988d15a941735,Garden
1992,Telheiras,38.760556,-9.165000,Parque Prof Prado Coelho,38.765785,-9.171544,4bf58dd8d48988d163941735,Park


Let's have a look at the __museums__.

We can visit History or Art Museums.

In [129]:
lx_venues_new[lx_venues_new['Metro Station'].isin(cluster2) & lx_venues_new['Venue Category'].isin(museums)]

,Metro Station,Metro Station Latitude,Metro Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category Id,Venue Category
654,Campo Grande,38.760000,-9.157778,Museu de Lisboa,38.758836,-9.156223,4bf58dd8d48988d190941735,History Museum
661,Campo Grande,38.760000,-9.157778,Museu Mundo Sporting,38.760821,-9.160870,4bf58dd8d48988d190941735,History Museum
711,Campo Grande,38.760000,-9.157778,Museu Rafael Bordalo Pinheiro,38.758865,-9.154120,4bf58dd8d48988d18f941735,Art Museum
1561,Roma,38.748889,-9.141389,Culturgest,38.740828,-9.142939,4bf58dd8d48988d1e2931735,Art Gallery
1774,Saldanha,38.735278,-9.144167,Culturgest,38.740828,-9.142939,4bf58dd8d48988d1e2931735,Art Gallery
1779,Saldanha,38.735278,-9.144167,Casa-Museu Dr. Anastácio Gonçalves,38.732594,-9.146419,4bf58dd8d48988d1e2931735,Art Gallery
1806,Saldanha,38.735278,-9.144167,Museu Calouste Gulbenkian,38.737812,-9.153749,4bf58dd8d48988d18f941735,Art Museum
1923,Telheiras,38.760556,-9.165000,Museu Mundo Sporting,38.760821,-9.160870,4bf58dd8d48988d190941735,History Museum
1936,Telheiras,38.760556,-9.165000,Museu de Lisboa,38.758836,-9.156223,4bf58dd8d48988d190941735,History Museum
1982,Telheiras,38.760556,-9.165000,Museu Rafael Bordalo Pinheiro,38.758865,-9.154120,4bf58dd8d48988d18f941735,Art Museum


### Cluster 3 - Outdoors

Cluster three has 5 metro stations, __Alameda__, __Anjos__, __Arroios__, __Intendente__ and __Sao Sebastiao__. As it happened with cluster 2, there is not the same variety in terms of Ice Cream Shops as in cluster 1, but there is a __concentration of Outdoors__.

In [100]:
lx_venues_new_grouped.loc[lx_venues_new_grouped['Cluster Labels'] == 2]

,Cluster Labels,Metro Station,Metro Station Latitude,Metro Station Longitude,Amphitheater,Aquarium,Art Gallery,Art Museum,Beach,Castle,...,History Museum,Ice Cream Shop,Museum,Park,Scenic Lookout,Science Museum,Sculpture Garden,Total Outdoors,Total Museums,Total
0,2,Alameda,38.736667,-9.133889,0,0,1,0,0,0,...,0,3,0,1,1,0,0,5,1,9
2,2,Anjos,38.726111,-9.134722,0,0,0,0,0,0,...,0,1,0,1,3,0,0,6,0,7
4,2,Arroios,38.733056,-9.133889,0,0,0,0,1,0,...,0,3,0,1,2,0,0,5,0,8
9,2,Intendente,38.722500,-9.135000,0,0,0,0,0,0,...,0,1,0,1,3,0,0,6,0,7
16,2,Sao Sebastiao,38.735000,-9.152222,1,0,0,2,0,0,...,0,1,1,2,1,0,1,6,3,10


Let's have a look at the __ice cream shops__.

Alameda and Arroios have a bit more variety and two of the shops are the same (Arroios and Alameda are very close).

In [130]:
cluster3 = lx_venues_new_grouped.loc[lx_venues_new_grouped['Cluster Labels'] == 2]['Metro Station'].tolist()
lx_venues_new[lx_venues_new['Metro Station'].isin(cluster3) & lx_venues_new['Venue Category'].isin(['Ice Cream Shop'])]

,Metro Station,Metro Station Latitude,Metro Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category Id,Venue Category
234,Alameda,38.736667,-9.133889,La Fabrica,38.736899,-9.141959,4bf58dd8d48988d1c9941735,Ice Cream Shop
235,Alameda,38.736667,-9.133889,Surf Gelados,38.739372,-9.137052,4bf58dd8d48988d1c9941735,Ice Cream Shop
238,Alameda,38.736667,-9.133889,FIB - il vero gelato italiano (geladosfib),38.744250,-9.134210,4bf58dd8d48988d1c9941735,Ice Cream Shop
319,Anjos,38.726111,-9.134722,Mú Gelato Italiano,38.722395,-9.140521,4bf58dd8d48988d1c9941735,Ice Cream Shop
486,Arroios,38.733056,-9.133889,La Fabrica,38.736899,-9.141959,4bf58dd8d48988d1c9941735,Ice Cream Shop
504,Arroios,38.733056,-9.133889,Fábrica do Gelado,38.724752,-9.132987,4bf58dd8d48988d1c9941735,Ice Cream Shop
518,Arroios,38.733056,-9.133889,Surf Gelados,38.739372,-9.137052,4bf58dd8d48988d1c9941735,Ice Cream Shop
1024,Intendente,38.722500,-9.135000,Mú Gelato Italiano,38.722395,-9.140521,4bf58dd8d48988d1c9941735,Ice Cream Shop
1909,Sao Sebastiao,38.735000,-9.152222,Häagen-Dazs,38.732756,-9.152862,4bf58dd8d48988d1c9941735,Ice Cream Shop


Let's have a look at the __outdoors__.

There is a mix of Gardens and Parks, but also a beach and an amphitheatre!

In [131]:
lx_venues_new[lx_venues_new['Metro Station'].isin(cluster3) & lx_venues_new['Venue Category'].isin(outdoors)]

,Metro Station,Metro Station Latitude,Metro Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category Id,Venue Category
200,Alameda,38.736667,-9.133889,Parque da Fonte Luminosa,38.737068,-9.132833,4bf58dd8d48988d163941735,Park
217,Alameda,38.736667,-9.133889,Miradouro da Penha de França,38.730900,-9.131145,4bf58dd8d48988d165941735,Scenic Lookout
226,Alameda,38.736667,-9.133889,Jardim Fernando Pessa,38.743069,-9.137231,4bf58dd8d48988d15a941735,Garden
243,Alameda,38.736667,-9.133889,Jardim do Arco do Cego,38.735908,-9.142256,4bf58dd8d48988d15a941735,Garden
283,Alameda,38.736667,-9.133889,Jardim Constantino,38.731012,-9.137064,4bf58dd8d48988d15a941735,Garden
301,Anjos,38.726111,-9.134722,Miradouro do Monte Agudo,38.726119,-9.131500,4bf58dd8d48988d165941735,Scenic Lookout
320,Anjos,38.726111,-9.134722,Miradouro da Penha de França,38.730900,-9.131145,4bf58dd8d48988d165941735,Scenic Lookout
328,Anjos,38.726111,-9.134722,Miradouro da Senhora do Monte,38.719072,-9.132793,4bf58dd8d48988d165941735,Scenic Lookout
336,Anjos,38.726111,-9.134722,Campo dos Mártires da Pátria,38.721630,-9.139917,4bf58dd8d48988d163941735,Park
339,Anjos,38.726111,-9.134722,Jardim do Torel,38.718980,-9.141523,4bf58dd8d48988d15a941735,Garden


### Cluster 4 - Ice Cream

Cluster four has 6 metro stations, __Areeiro__, __Baixa-Chiado__, __Cabo Ruivo__, __Cais do Sodre__, __Moscavide__ and __Oriente__. There is a good variety of Ice Cream Shops, and a good mix of __Outdoors__ and __Museums__.

In [101]:
lx_venues_new_grouped.loc[lx_venues_new_grouped['Cluster Labels'] == 3]

,Cluster Labels,Metro Station,Metro Station Latitude,Metro Station Longitude,Amphitheater,Aquarium,Art Gallery,Art Museum,Beach,Castle,...,History Museum,Ice Cream Shop,Museum,Park,Scenic Lookout,Science Museum,Sculpture Garden,Total Outdoors,Total Museums,Total
3,3,Areeiro,38.742222,-9.134167,0,0,1,0,0,0,...,0,4,0,1,0,0,0,3,1,8
5,3,Baixa-Chiado,38.710556,-9.139444,0,0,0,0,0,0,...,1,6,0,0,1,0,0,1,1,8
6,3,Cabo Ruivo,38.762500,-9.103889,0,1,0,0,0,0,...,0,3,0,0,1,1,0,2,2,7
7,3,Cais do Sodre,38.705278,-9.142500,0,0,1,0,0,0,...,0,4,0,0,1,0,0,1,1,6
11,3,Moscavide,38.775000,-9.102500,0,0,0,0,0,0,...,0,3,0,1,2,0,0,3,0,6
12,3,Oriente,38.767222,-9.100278,0,1,0,0,0,0,...,0,4,0,1,1,1,0,2,2,8


Let's have a look at the __ice cream shops__.

There are a few ice cream shops shared between Baixa-Chiado and Cais do Sodre. The same happens with Oriente, Moscavide and Cabo Ruivo.

In [132]:
cluster4 = lx_venues_new_grouped.loc[lx_venues_new_grouped['Cluster Labels'] == 3]['Metro Station'].tolist()
lx_venues_new[lx_venues_new['Metro Station'].isin(cluster4) & lx_venues_new['Venue Category'].isin(['Ice Cream Shop'])]

,Metro Station,Metro Station Latitude,Metro Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category Id,Venue Category
0,Areeiro,38.742222,-9.134167,FIB - il vero gelato italiano (geladosfib),38.744250,-9.134210,4bf58dd8d48988d1c9941735,Ice Cream Shop
11,Areeiro,38.742222,-9.134167,Casa do Gelado,38.744872,-9.139545,4bf58dd8d48988d1c9941735,Ice Cream Shop
37,Areeiro,38.742222,-9.134167,Surf Gelados,38.739372,-9.137052,4bf58dd8d48988d1c9941735,Ice Cream Shop
51,Areeiro,38.742222,-9.134167,La Fabrica,38.736899,-9.141959,4bf58dd8d48988d1c9941735,Ice Cream Shop
534,Baixa-Chiado,38.710556,-9.139444,Santini,38.711138,-9.139668,4bf58dd8d48988d1c9941735,Ice Cream Shop
536,Baixa-Chiado,38.710556,-9.139444,Amorino Chiado,38.710808,-9.140747,4bf58dd8d48988d1c9941735,Ice Cream Shop
545,Baixa-Chiado,38.710556,-9.139444,Amorino,38.711807,-9.138144,4bf58dd8d48988d1c9941735,Ice Cream Shop
559,Baixa-Chiado,38.710556,-9.139444,Grom,38.710913,-9.140544,4bf58dd8d48988d1c9941735,Ice Cream Shop
586,Baixa-Chiado,38.710556,-9.139444,Fragoleto,38.709725,-9.136435,4bf58dd8d48988d1c9941735,Ice Cream Shop
621,Baixa-Chiado,38.710556,-9.139444,Gelato Therapy,38.710149,-9.135339,4bf58dd8d48988d1c9941735,Ice Cream Shop


Let's have a look at the __outdoors__.

There is a mix of Gardens, Parks, Scenic Lookouts and also a Farm.

In [133]:
lx_venues_new[lx_venues_new['Metro Station'].isin(cluster4) & lx_venues_new['Venue Category'].isin(outdoors)]

,Metro Station,Metro Station Latitude,Metro Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category Id,Venue Category
2,Areeiro,38.742222,-9.134167,Jardim Fernando Pessa,38.743069,-9.137231,4bf58dd8d48988d15a941735,Garden
6,Areeiro,38.742222,-9.134167,Parque da Fonte Luminosa,38.737068,-9.132833,4bf58dd8d48988d163941735,Park
61,Areeiro,38.742222,-9.134167,Jardim do Arco do Cego,38.735908,-9.142256,4bf58dd8d48988d15a941735,Garden
558,Baixa-Chiado,38.710556,-9.139444,Terraços do Carmo,38.712332,-9.139709,4bf58dd8d48988d165941735,Scenic Lookout
780,Cabo Ruivo,38.762500,-9.103889,Quinta Pedagógica Olivais,38.763139,-9.112649,4bf58dd8d48988d15b941735,Farm
796,Cabo Ruivo,38.762500,-9.103889,Esplanada Vasco da Gama,38.767651,-9.096164,4bf58dd8d48988d165941735,Scenic Lookout
962,Cais do Sodre,38.705278,-9.142500,Miradouro de Santa Catarina (Adamastor),38.709467,-9.147659,4bf58dd8d48988d165941735,Scenic Lookout
1237,Moscavide,38.775000,-9.102500,Torre Vasco da Gama,38.774959,-9.091986,4bf58dd8d48988d165941735,Scenic Lookout
1242,Moscavide,38.775000,-9.102500,Esplanada Vasco da Gama,38.767651,-9.096164,4bf58dd8d48988d165941735,Scenic Lookout
1273,Moscavide,38.775000,-9.102500,Jardim Almeida Garrett,38.781842,-9.109704,4bf58dd8d48988d163941735,Park


Let's have a look at the __museums__.

We can visit History, Art or Science Museums and also the Aquarium.

In [134]:
lx_venues_new[lx_venues_new['Metro Station'].isin(cluster4) & lx_venues_new['Venue Category'].isin(museums)]

,Metro Station,Metro Station Latitude,Metro Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category Id,Venue Category
24,Areeiro,38.742222,-9.134167,Culturgest,38.740828,-9.142939,4bf58dd8d48988d1e2931735,Art Gallery
546,Baixa-Chiado,38.710556,-9.139444,Museu Arqueológico do Carmo,38.711980,-9.140773,4bf58dd8d48988d190941735,History Museum
776,Cabo Ruivo,38.762500,-9.103889,Pavilhão do Conhecimento - Ciência Viva,38.762770,-9.095536,4bf58dd8d48988d191941735,Science Museum
782,Cabo Ruivo,38.762500,-9.103889,Oceanário de Lisboa,38.763370,-9.093719,4fceea171983d5d06c3e9823,Aquarium
878,Cais do Sodre,38.705278,-9.142500,Montana Lisboa - Underdogs Public Art Store,38.705402,-9.144307,4bf58dd8d48988d1e2931735,Art Gallery
1396,Oriente,38.767222,-9.100278,Pavilhão do Conhecimento - Ciência Viva,38.762770,-9.095536,4bf58dd8d48988d191941735,Science Museum
1403,Oriente,38.767222,-9.100278,Oceanário de Lisboa,38.763370,-9.093719,4fceea171983d5d06c3e9823,Aquarium


## Conclusion 

The objective of this project was to find what is the best metro station to stop in Lisbon (Portugal) if someone wants to go for an ice cream and then go either to an outdoor space or a instead to a museum.

In order to do that, I have located all the venues that are with a range of 1Km of each metro station, using FourSquare. Due to limitations, my subset was limited to the Red and Green Metro Lines.

I have filtered the data set to __Ice Cream Shops__, __Museums__ and __Outdoors__ and I have calculated the subtotal of each of these categories of venues for each metro station.

After that, I have identified 4 different clusters of Metro Stations using __KMeans__:


1. If the __Ice Cream Shop__ is the priority then __Cluster 1__ is definitely the best choice, because there is a good range of different shops. Other than that, it is very close to the Castle, and it is the cluster that shows the highest number of venues of all four clusters.

    * Martim Moniz
    * Rossio

2. If going to the Castle is not in your plans, then you can also go with __Cluster 4__, that also has a good range of __Ice Cream Shops__ and a good mix between __Outdoors__ and __Museums__.

    * Areeiro
    * Baixa-Chiado
    * Cabo Ruivo
    * Cais do Sodre
    * Moscavide
    * Oriente

3. If you are more interested in the options after getting your ice cream, and you are trying to avoid the more touristic areas, then __Cluster 2__ is good for you.
    
      * Alvalade
      * Campo Grande
      * Roma
      * Saldanha
      * Telheiras

4. If the most important thing is to have a good variety of __Outdoors__ than go to any of the stations in __Cluster 3__.

    * Alameda
    * Anjos
    * Arroios
    * Intendente
    * Sao Sebastiao